In [ ]:
# 1. create a test set
    # sample ~50 video clips @ 25 fps
    # manually record clock start/stop timestamps
# 2. record pipeline error

In [ ]:
import os
import ffmpeg
import random

# get video file paths
vids_dir = '/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/data-sources/nba/data/replays'
vid_fps = [os.path.join(vids_dir, f) for f in os.listdir(vids_dir) if os.path.isfile(os.path.join(vids_dir, f))]
vid_fps  = [v for v in vid_fps if v.endswith('.mp4')][0: 50]
random.shuffle(vid_fps)

In [ ]:
import os
import random
import subprocess
from pathlib import Path

def sample_video_clips(video_paths, output_dir, clip_duration=10, fps=25):
    
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    for video_path in video_paths:
        result = subprocess.run(
            ["ffmpeg", "-i", video_path, "-hide_banner"],
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE
        )
        duration_line = [x for x in result.stderr.decode().split('\n') if "Duration" in x]
        if duration_line:
            duration_str = duration_line[0].split(',')[0].split('Duration:')[1].strip()
            h, m, s = map(float, duration_str.split(':'))
            total_duration = h * 3600 + m * 60 + s
        else:
            print(f"Could not determine duration of video: {video_path}")
            continue
        if total_duration > clip_duration:
            start_time = random.uniform(0, total_duration - clip_duration)
        else:
            print(f"Video is shorter than the clip duration: {video_path}")
            continue

        video_name = os.path.basename(video_path)
        output_path = os.path.join(output_dir, f"clip_{video_name}")

        ffmpeg_command = [
            "ffmpeg",
            "-i", video_path,
            "-ss", str(start_time),
            "-t", str(clip_duration),
            "-r", str(fps),
            "-c", "copy",
            output_path
        ]

        subprocess.run(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Saved clip to {output_path}")

output_directory = "/playpen-storage/levlevi/nba-positions-videos-dataset/testing/quantitative-benchmark/test-set"
sample_video_clips(vid_fps, output_directory)